In [ ]:
# pip install openmeteo-requests

In [ ]:
# pip install requests-cache retry-requests numpy pandas

In [3]:
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=3600)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# List of coordinates (latitude and longitude) for each MA county
ma_counties_coordinates = [
    {"county_name": "Barnstable", "latitude": 41.7003, "longitude": -70.3002},
    {"county_name": "Berkshire", "latitude": 42.3118, "longitude": -73.1822},
    {"county_name": "Bristol", "latitude": 41.7938, "longitude": -71.1350},
    {"county_name": "Dukes", "latitude": 41.4033, "longitude": -70.6693},
    {"county_name": "Essex", "latitude": 42.6334, "longitude": -70.7829},
    {"county_name": "Franklin", "latitude": 42.5795, "longitude": -72.6151},
    {"county_name": "Hampden", "latitude": 42.1175, "longitude": -72.6009},
    {"county_name": "Hampshire", "latitude": 42.3389, "longitude": -72.6417},
    {"county_name": "Middlesex", "latitude": 42.4672, "longitude": -71.2874},
    {"county_name": "Nantucket", "latitude": 41.2835, "longitude": -70.0995},
    {"county_name": "Norfolk", "latitude": 42.1621, "longitude": -71.1912},
    {"county_name": "Plymouth", "latitude": 41.9880, "longitude": -70.7528},
    {"county_name": "Suffolk", "latitude": 42.3601, "longitude": -71.0589},
    {"county_name": "Worcester", "latitude": 42.4002, "longitude": -71.9065}
]

# Dictionary to store weather data by county
county_weather_data = {}

# Function to fetch and process weather data for a given set of coordinates
def fetch_weather_data(county_name, latitude, longitude):
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "current": ["temperature_2m", "relative_humidity_2m", "precipitation", "weather_code", "cloud_cover", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m"],
	    "daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "sunrise", "sunset", "uv_index_max", "precipitation_probability_max", "wind_speed_10m_max", "wind_gusts_10m_max"],
        "timezone": "America/New_York",
        "forecast_days": 7
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process the response for the given coordinates
    response = responses[0]

    # Process current data
    current = response.Current()
    current_data = [{
        "date": pd.to_datetime(current.Time(), unit="s", utc=True),
        "county": county_name,
        "latitude": latitude,
        "longitude": longitude,
        "temperature_2m": current.Variables(0).ValuesAsNumpy(),
        "relative_humidity_2m": current.Variables(1).ValuesAsNumpy(),
        "precipitation": current.Variables(2).ValuesAsNumpy(),
        "weather_code": current.Variables(3).ValuesAsNumpy(),
        "cloud_cover": current.Variables(4).ValuesAsNumpy(),
        "wind_speed_10m": current.Variables(5).ValuesAsNumpy(),
        "wind_direction_10m": current.Variables(6).ValuesAsNumpy(),
        "wind_gusts_10m": current.Variables(7).ValuesAsNumpy()
    }]
    current_df = pd.DataFrame.from_records(current_data, index='date')


    # Process daily data
    daily = response.Daily()
    daily_data = {
        "date": pd.date_range(
            start=pd.to_datetime(daily.Time(), unit="s", utc=True),
            end=pd.to_datetime(daily.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=daily.Interval()),
            inclusive="left"
        ),
        "county": county_name,
        "latitude": latitude,
        "longitude": longitude,
        "weather_code": daily.Variables(0).ValuesAsNumpy(),
        "temperature_2m_max": daily.Variables(1).ValuesAsNumpy(),
        "temperature_2m_min": daily.Variables(2).ValuesAsNumpy(),
        "sunrise": daily.Variables(3).ValuesAsNumpy(),
        "sunset": daily.Variables(4).ValuesAsNumpy(),
        "uv_index_max": daily.Variables(5).ValuesAsNumpy(),
        "precipitation_probability_max": daily.Variables(6).ValuesAsNumpy(),
        "wind_speed_10m_max": daily.Variables(7).ValuesAsNumpy(),
        "wind_gusts_10m_max": daily.Variables(8).ValuesAsNumpy()
    }
    daily_df = pd.DataFrame(daily_data)

    # Store DataFrames in the dictionary by county
    county_weather_data[county_name] = {
        "current": current_df,
        "daily": daily_df
    }

# Loop through each set of coordinates and fetch weather data
for county in ma_counties_coordinates:
    fetch_weather_data(county["county_name"], county["latitude"], county["longitude"])

In [5]:
# Create a directory to store CSV files
output_dir = 'dataset/county_weather_data/'
os.makedirs(output_dir, exist_ok=True)

# Iterate through counties and data types
for county_name, data_types in county_weather_data.items():
    for data_type, data in data_types.items():
        # Create DataFrame from the data
        df = pd.DataFrame(data)

        # Construct CSV file path
        csv_file_path = os.path.join(output_dir, f"{county_name}_{data_type}.csv")

        # Export to CSV
        df.to_csv(csv_file_path, index=False)

        print(f"Data for {county_name} ({data_type}) saved to {csv_file_path}")

print("All data exported to CSV files.")

Data for Barnstable (current) saved to dataset/county_weather_data/Barnstable_current.csv
Data for Barnstable (daily) saved to dataset/county_weather_data/Barnstable_daily.csv
Data for Berkshire (current) saved to dataset/county_weather_data/Berkshire_current.csv
Data for Berkshire (daily) saved to dataset/county_weather_data/Berkshire_daily.csv
Data for Bristol (current) saved to dataset/county_weather_data/Bristol_current.csv
Data for Bristol (daily) saved to dataset/county_weather_data/Bristol_daily.csv
Data for Dukes (current) saved to dataset/county_weather_data/Dukes_current.csv
Data for Dukes (daily) saved to dataset/county_weather_data/Dukes_daily.csv
Data for Essex (current) saved to dataset/county_weather_data/Essex_current.csv
Data for Essex (daily) saved to dataset/county_weather_data/Essex_daily.csv
Data for Franklin (current) saved to dataset/county_weather_data/Franklin_current.csv
Data for Franklin (daily) saved to dataset/county_weather_data/Franklin_daily.csv
Data for